In [ ]:
## https://www.youtube.com/watch?v=SSW9LzOJSus

### Install and Load Dependencies 

In [ ]:
# %%capture
# !pip install opencv-python tensorflow==2.4.1 tensorflow-gpu==2.4.1 matplotlib

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

# dependencies for colab webcam connection
# from IPython.display import display, Javascript, Image
# from google.colab.output import eval_js
# from base64 import b64decode, b64encode
# import PIL
# import io
# import html
# import time

In [9]:
import urllib.request

url_lightning_model = "https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/3?lite-format=tflite"
url_thunder_model = "https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/3?lite-format=tflite"

urllib.request.urlretrieve(url_thunder_model, "model/model.tflite")

('model/model.tflite', <http.client.HTTPMessage at 0x2d980089a80>)

### Load Model

In [10]:
interpreter = tf.lite.Interpreter(model_path='model/model.tflite')
interpreter.allocate_tensors()

### Make Detections

In [11]:
cap = cv2.VideoCapture("data\customer_video\Standing and sitting-frontal plane.mp4")

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  
  if ret == True:
 
    # Display the resulting frame
    cv2.imshow('Frame',frame)

    # reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 256, 256)
    input_img = tf.cast(img, dtype=tf.float32)

    # setup input & output
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # make predictions
    interpreter.set_tensor(input_details[0]['index'], np.array(input_img))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    # print(keypoints_with_scores)

    # render
    draw_connections(frame, keypoints_with_scores, EDGES, 0.2)
    draw_keypoints(frame, keypoints_with_scores, 0.2)

    cv2.imshow('Movenet', frame)

    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
 
  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()
# Closes all the frames
cv2.destroyAllWindows()

In [15]:
# cv2.imwrite("test.jpg", img) # write image

# out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

### Draw Keypoints

In [5]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

### Draw Edges

In [6]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [7]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1] # get coordinates
        y2, x2, c2 = shaped[p2]

        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255,0,0), 2)